## Data Analytics Module 2 - Apache Spark Manipulations

### Loading CSV files into the databricks file system (dbfs)

In [ ]:
%sh
rm -r /dbfs/spark_lab
mkdir /dbfs/spark_lab
wget -O /dbfs/spark_lab/2019.csv https://raw.githubusercontent.com/kuljotSB/DatabricksUdemyCourse/refs/heads/main/DataAnalytics/2019_edited.csv
wget -O /dbfs/spark_lab/2020.csv https://raw.githubusercontent.com/kuljotSB/DatabricksUdemyCourse/refs/heads/main/DataAnalytics/2020_edited.csv
wget -O /dbfs/spark_lab/2021.csv https://raw.githubusercontent.com/kuljotSB/DatabricksUdemyCourse/refs/heads/main/DataAnalytics/2021_edited.csv
     

### Loading the CSV files into a dataframe

In [ ]:
df = spark.read.load('spark_lab/*.csv', format='csv')
display(df.limit(100))

### Defining Schema for the Dataframe

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
orderSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("Email", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("Tax", FloatType())
])
df = spark.read.load('/spark_lab/*.csv', format='csv', schema=orderSchema)
display(df.limit(100))
     

### Cleaning the Data

In [ ]:
from pyspark.sql.functions import col
df = df.dropDuplicates()
df = df.withColumn('Tax', col('UnitPrice') * 0.08)
df = df.withColumn('Tax', col('Tax').cast("float"))

### Creating a new Dataframe

In [ ]:
customers_df = df.select("CustomerName", "Email", "Item", "Quantity")
customers_df = customers_df.withColumn("FirstName", split(customers_df["CustomerName"], " ").getItem(0))
customers_df = customers_df.withColumn("LastName", split(customers_df["CustomerName"], " ").getItem(1))

display(customers_df)

### Creating distinct customer entries

In [ ]:
print(customers_df.count())
print(customers_df.distinct().count())

### Creating a Product Sales Dataframe

In [ ]:
productSales = df.select("Item", "Quantity").groupBy("Item").sum()
display(productSales)

### Aggregating Yearly Sales

In [ ]:
yearlySales = df.select(year("OrderDate").alias("Year")).groupBy("Year").count().orderBy("Year")
display(yearlySales)